In [ ]:
## Imports  
import numpy as np
import pandas as pd
import sys
import os
import time
import pickle
import tensorflow as tf
import time
from importlib import reload
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras.callbacks import LearningRateScheduler

from models.one_hot_drugs_models import build_CNN_Phos_LM 
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from scripts.data_wrangling import import_phos
from scripts.feature_selection import fs_landmark_targets, true_phosphosite_filter
from scripts.data_preparation import data_prep
from scripts.data_selection import data_indexing
from scripts.model_training import train_model, train_model_ML, scheduler, split_all_scores

print('Imports: Done')
tf.keras.utils.set_random_seed(524) # 524, 270, 345

## Data Wrangling, feature selection and preparation

# Import and wrangle phosphoproteomics and GDSC1 drug data
phos_df, drug_df, drug_matrix, _all_cls, _all_drugs, common_ind  = import_phos()
# Create list of phosphosites filtered by feature selection method
phosphosites = fs_landmark_targets(phos_df)
# Reduce remaining features down by feature selection
phos_df = phos_df.filter(phosphosites,axis=1)
# Create dataframes for omics, drugs and target values
x_drug, x_all, y_series = data_prep(drug_df,phos_df,common_ind)
# import stored training and testing pairs 
with open (f'train_test_pairs/_train_test_pairs_1.ob', 'rb') as fp:
    train_test_list = pickle.load(fp)

## Training model and testing

all_scores = []
trials = len(train_test_list) # number of train/test splits tested 
epochs = 50 # max training epochs (50)

for i in range(trials):
    print(f"\nTrial {i+1}\n")
    
    # select train and test pairs
    train_pairs = train_test_list[i][0]
    test_pairs  = train_test_list[i][1]
    # index for each set of train-test pairs
    xo_train, xd_train, y_train, xo_test, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all,x_drug,y_series) # select train and test

    m_func = build_CNN_Phos_LM(xo_train,xd_train) # set model and parameters
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler

    scores = train_model(m_func,lr_scheduler, 
                         x_train = [xo_train, xd_train], x_test = [xo_test, xd_test], 
                         y_train = y_train, y_test = y_test, 
                         epochs = epochs)
    
    all_scores.append(scores)
    
# split scores into individual lists
r2_scores, mse_scores, pearson_scores = split_all_scores(all_scores)

# find mean of all scores    
print('\nmean r2: ',np.mean(r2_scores))
print('\nr2 scores:')
print(*r2_scores, sep = '\n')

# find mean of all scores    
print('\nmean mse: ',np.mean(mse_scores))
print('\nr2 scores:')
print(*mse_scores, sep = '\n')

# find mean of all scores    
print('\nmean pearson: ',np.mean(pearson_scores))
print('\nr2 scores:')
print(*pearson_scores, sep = '\n')